In [80]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import csv
import time
from selenium.common.exceptions import StaleElementReferenceException

# 웹 드라이버 설정
chrome_driver_path = "./chromedriver.exe"  # ChromeDriver 경로를 지정하세요.
options = Options()
options.add_argument("--headless")  # 브라우저 창 숨기기
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Chrome WebDriver 초기화
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)

In [63]:
driver.get("https://www.koreabaseball.com/Record/Player/PitcherBasic/BasicOld.aspx")

import time
import csv
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 기존 pitcher 데이터를 읽어옴
existing_pitchers = set()
with open('pitcher.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader)  # 헤더는 건너뛰기
    for row in reader:
        existing_pitchers.add(row[1])  # 선수명만 set에 저장

# o_pitcher.csv 파일을 열고 데이터 저장
with open('o_pitcher.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['팀명', '선수명'])  # 헤더 작성

    # 팀 목록 설정 (option 태그에서 value 값)
    team_options = ['HT', 'SS', 'LG', 'OB', 'KT', 'SK', 'LT', 'HH', 'NC', 'WO']

    # 드롭다운을 새로 찾고, 다시 선택하도록 수정
    year_dropdown = Select(driver.find_element(By.ID, 'cphContents_cphContents_cphContents_ddlSeason_ddlSeason'))
    season_dropdown = Select(driver.find_element(By.ID, 'cphContents_cphContents_cphContents_ddlSeries_ddlSeries'))
    team_dropdown = Select(driver.find_element(By.ID, 'cphContents_cphContents_cphContents_ddlTeam_ddlTeam'))

    # 팀을 먼저 선택하고, 각 팀에 대해 2019~2023년 데이터를 반복
    for team_value in team_options:
        print(f"팀: {team_value} 선택 시작")  # 디버깅용
        # 드롭다운에서 팀을 선택
        team_dropdown = Select(driver.find_element(By.ID, 'cphContents_cphContents_cphContents_ddlTeam_ddlTeam'))
        team_dropdown.select_by_value(team_value)
        time.sleep(2)

        # 시즌 드롭다운에서 정규시즌 선택
        season_dropdown = Select(driver.find_element(By.ID, 'cphContents_cphContents_cphContents_ddlSeries_ddlSeries'))
        season_dropdown.select_by_value('0')  # 정규시즌
        time.sleep(2)

        # 2019~2023년 데이터를 선택하고 반복
        for year in range(2019, 2024):  # 2019~2023년
            year_dropdown = Select(driver.find_element(By.ID, 'cphContents_cphContents_cphContents_ddlSeason_ddlSeason'))
            year_dropdown.select_by_value(str(year))
            time.sleep(2)

            # 페이징 처리
            while True:
                print(f"팀: {team_value}, 연도: {year} - 페이지 데이터 수집 시작")  # 디버깅용
                
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.tData01 tbody tr')))
                table_rows = driver.find_elements(By.CSS_SELECTOR, '.tData01 tbody tr')

                for row in table_rows:
                    columns = row.find_elements(By.TAG_NAME, 'td')
                    if len(columns) > 0:
                        team_name = columns[2].text  # 팀명 (3번째 열)
                        player_name = columns[1].text  # 선수명 (2번째 열)

                        # 기존 pitcher와 o_pitcher에서 중복되는 선수는 건너뛰기
                        if player_name not in existing_pitchers:
                            writer.writerow([team_name, player_name])  # 새로운 선수만 추가
                            print(f"선수 추가: {player_name} ({team_name})")  # 디버깅용     

                # 페이징 버튼 확인
                paging_buttons = driver.find_elements(By.CSS_SELECTOR, '.paging a')
                current_page = driver.find_element(By.CSS_SELECTOR, '.paging a.on').text  # 현재 페이지 번호

                # 다음 페이지 버튼 찾기
                next_page_button = None
                for button in paging_buttons:
                    if button.text.isdigit() and int(button.text) == int(current_page) + 1:
                        next_page_button = button
                        break

                # 다음 페이지로 이동하거나 종료
                if next_page_button:
                    print(f"다음 페이지로 이동: {current_page} -> {int(current_page) + 1}")  # 디버깅용
                    driver.execute_script("arguments[0].click();", next_page_button)
                    time.sleep(2)
                   
                else:
                    print("더 이상 다음 페이지가 없습니다.")  # 디버깅용
                    # 첫 번째 페이지로 돌아가기
                    first_page_button = driver.find_element(By.CSS_SELECTOR, '.paging a:first-child')  # 첫 번째 페이지 버튼
                    driver.execute_script("arguments[0].click();", first_page_button)
                    time.sleep(2)
                    break
                    
                
# 작업이 끝난 후 브라우저 닫기
driver.quit()

팀: HT 선택 시작
팀: HT, 연도: 2019 - 페이지 데이터 수집 시작
선수 추가: 문경찬 (KIA)
선수 추가: 임기준 (KIA)
선수 추가: 고영창 (KIA)
선수 추가: 박진태 (KIA)
선수 추가: 차명진 (KIA)
선수 추가: 윌랜드 (KIA)
선수 추가: 하준영 (KIA)
선수 추가: 터너 (KIA)
선수 추가: 김세현 (KIA)
선수 추가: 황인준 (KIA)
선수 추가: 김윤동 (KIA)
선수 추가: 양승철 (KIA)
선수 추가: 김승범 (KIA)
더 이상 다음 페이지가 없습니다.
팀: HT, 연도: 2020 - 페이지 데이터 수집 시작
선수 추가: 황윤호 (KIA)
선수 추가: 브룩스 (KIA)
선수 추가: 변시원 (KIA)
선수 추가: 남하준 (KIA)
선수 추가: 가뇽 (KIA)
선수 추가: 황인준 (KIA)
선수 추가: 홍상삼 (KIA)
선수 추가: 김명찬 (KIA)
선수 추가: 고영창 (KIA)
선수 추가: 양승철 (KIA)
선수 추가: 서덕원 (KIA)
선수 추가: 김승범 (KIA)
선수 추가: 김현준 (KIA)
선수 추가: 박진태 (KIA)
선수 추가: 차명진 (KIA)
더 이상 다음 페이지가 없습니다.
팀: HT, 연도: 2021 - 페이지 데이터 수집 시작
선수 추가: 최정용 (KIA)
선수 추가: 황윤호 (KIA)
선수 추가: 브룩스 (KIA)
선수 추가: 멩덴 (KIA)
선수 추가: 홍상삼 (KIA)
선수 추가: 다카하시 (KIA)
선수 추가: 김현준 (KIA)
선수 추가: 고영창 (KIA)
선수 추가: 박진태 (KIA)
선수 추가: 서덕원 (KIA)
선수 추가: 차명진 (KIA)
선수 추가: 남하준 (KIA)
다음 페이지로 이동: 1 -> 2
팀: HT, 연도: 2021 - 페이지 데이터 수집 시작
선수 추가: 변시원 (KIA)
선수 추가: 강경학 (KIA)
더 이상 다음 페이지가 없습니다.
팀: HT, 연도: 2022 - 페이지 데이터 수집 시작
선수 추가: 고영창 (KIA)
선수 추가: 놀린 (KIA)
선수 추가: 

In [81]:
driver.get("https://statiz.sporki.com/")
# pitcher.csv 파일에서 선수 목록을 읽어오기
with open('old_pitcher.csv', mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader)  # 헤더 건너뛰기
    players = list(reader)
    
    # pitcher_stat.csv 파일을 새로 생성하여 데이터를 기록하기 위해 열기
    with open('old_pitcher_stats.csv', mode='a', newline='', encoding='utf-8') as stat_file:
        writer = csv.writer(stat_file)
        
        # 헤더 추가
        writer.writerow([
            '현재 소속팀', '선수명', '연도', '당시 소속팀', '나이', '출장', '선발', '구원', '경기 마무리', '완투', '완봉', 
            '승', '패', '세이브', '홀드', '이닝', '자책점', '실점', '책임실점', '상대한 타자 수', '안타',
            '2루타', '3루타', '홈런', '4구', '사구', '고의4구', '삼진', '실책 출루 허용 횟수', '보크', '폭투', 
            'ERA', '9이닝 당 실점', '9이닝 당 책임실점', '수비 무관 평균자책점', '이닝 당 출루 허용률', '대체 선수 대비 승리 기여도', '연봉'
        ])
        
        # 선수 목록을 하나씩 처리
        for team_name, player_name in players:
            try:
                # '선수 검색' 버튼 클릭
                search_button = driver.find_element(By.CSS_SELECTOR, '.new_searchPlayer .btn_box')
                search_button.click()
                
                # 선수 검색 모달창 로드 대기
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.popBox.modal8')))
                
                # 검색 입력창에 선수 이름 입력
                search_input = driver.find_element(By.ID, 's')
                search_input.clear()  # 기존 내용 지우기
                search_input.send_keys(player_name)  # 선수 이름 입력
                
                # 검색 버튼 클릭
                search_btn = driver.find_element(By.CSS_SELECTOR, '.popBox .search_box .btn')
                driver.execute_script("arguments[0].click();", search_btn)
                
                # 현재 URL 확인 (검색 결과 페이지인지 확인)
                current_url = driver.current_url
                
                # 검색 결과 페이지 확인
                if "m=search" in current_url:
                    
                    # 검색 결과 테이블 로드 대기
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.table_type01')))
                    
                    # 검색 결과 테이블 데이터 추출
                    result_rows = driver.find_elements(By.CSS_SELECTOR, '.table_type01 tbody tr')
                    
                    # 검색 결과가 없으면 건너뜀
                    if not result_rows:
                        print(f"{player_name} : 검색 결과 없음 (URL: {current_url})")
                        continue
                    
                    # 팀명과 일치하는 선수 검색
                    matched_players = []
                    for row in result_rows:
                        cols = row.find_elements(By.TAG_NAME, 'td')
                        if len(cols) > 1:
                            result_team = cols[3].text  # '최근 소속팀' 열
                            if result_team == team_name:
                                player_link = row.find_element(By.TAG_NAME, 'a')  # 선수 프로필 링크
                                matched_players.append(player_link)
                    
                    # 일치하는 결과가 없으면 건너뜀
                    if not matched_players:
                        print(f"{player_name} : 일치하는 팀 없음 (URL: {current_url})")
                        continue
                    
                    # 일치하는 결과가 여러 개인 경우 수동 처리
                    if len(matched_players) > 1:
                        print(f"{player_name} : 중복값 (URL: {current_url})")
                        continue
                    
                    # 일치하는 결과가 1개인 경우 처리
                    player_link = matched_players[0]
                    driver.execute_script("arguments[0].click();", player_link)
                    print(f"선수 {player_name} 중복 검색 완료")
                    
                # 연도별 통계 페이지 접근
                year_link = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.p_schedule')))
                driver.execute_script("arguments[0].click();", year_link)
                
                # 연도별 통계 테이블 로드 대기
                WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.table_type02')))
                
                table_body = driver.find_element(By.CSS_SELECTOR, '.table_type02 tbody')
                rows = table_body.find_elements(By.TAG_NAME, 'tr')
    
                if not rows:  # tbody가 비어있는 경우 처리
                    print(f"선수 {player_name}: 기록 없음.")
                    continue
                
                year_salary_data = []  # 연도별 통계 저장 리스트
                skip_salary = False  # 연봉 처리 여부를 위한 플래그
                
                # 연도별 통계 저장 작업
                for row in rows:
                    columns = row.find_elements(By.TAG_NAME, 'td')
                    
                    # 컬럼 개수가 35가 아니면 예외 처리
                    if len(columns) != 36:
                        print(f"선수 {player_name} 데이터 컬럼 불일치: {len(columns)}개")
                        writer.writerow([team_name, player_name] + [''] * 34)
                        skip_salary = True  # 연봉 처리 스킵
                        break
                        
                    if len(columns) > 1:  # 데이터가 있는 경우만 처리
                        # 각 컬럼 데이터 추출
                        year = columns[0].text.strip()  # 연도 추출
                        team_c = columns[1].text.strip() # 당시 소속팀
                        age = columns[2].text.strip()  # 나이
                        games = columns[4].text.strip()  # 출장
                        gs = columns[5].text.strip()  # 선발
                        gr = columns[6].text.strip()  # 구원
                        gf = columns[7].text.strip()  # 경기 마무리
                        cg = columns[8].text.strip()  # 완투
                        sho = columns[9].text.strip()  # 완봉
                        w = columns[10].text.strip()  # 승
                        l = columns[11].text.strip()  # 패
                        s = columns[12].text.strip()  # 세이브
                        hd = columns[13].text.strip()  # 홀드
                        ip = columns[14].text.strip()  # 이닝
                        er = columns[15].text.strip()  # 자책점
                        r = columns[16].text.strip()  # 실점
                        rra = columns[17].text.strip()  # 책임실점
                        tbf = columns[18].text.strip()  # 상대한 타자 수
                        h = columns[19].text.strip()  # 안타
                        _2b = columns[20].text.strip()  # 2루타
                        _3b = columns[21].text.strip()  # 3루타
                        hr = columns[22].text.strip()  # 홈런
                        bb = columns[23].text.strip()  # 볼넷
                        hp = columns[24].text.strip()  # 사구
                        ib = columns[25].text.strip()  # 고의4구
                        so = columns[26].text.strip()  # 삼진
                        roe = columns[27].text.strip()  # 실책 출루 허용 횟수
                        bk = columns[28].text.strip()  # 보크
                        wp = columns[29].text.strip()  # 폭투
                        era = columns[30].text.strip()  # ERA
                        ra9 = columns[31].text.strip()  # 9이닝 당 실점
                        rra9 = columns[32].text.strip()  # 9이닝 당 책임실점
                        fip = columns[33].text.strip()  # FIP
                        whip = columns[34].text.strip()  # WHIP
                        war = columns[35].text.strip()  # WAR
                        # 연도별 통계에 해당하는 데이터 추가
                        year_salary_data.append({
                            'year': year,
                            'team_c': team_c,
                            'age': age,
                            'games': games,
                            'gs': gs,
                            'gr': gr,
                            'gf': gf,
                            'cg': cg,
                            'sho': sho,
                            'w': w,
                            'l': l,
                            's': s,
                            'hd': hd,
                            'ip': ip,
                            'er': er,
                            'r': r,
                            'rra': rra,
                            'tbf': tbf,
                            'h': h,
                            '_2b': _2b,
                            '_3b': _3b,
                            'hr': hr,
                            'bb': bb,
                            'hp': hp,
                            'ib': ib,
                            'so': so,
                            'roe': roe,
                            'bk': bk,
                            'wp': wp,
                            'era': era,
                            'ra9': ra9,
                            'rra9': rra9,
                            'fip': fip,
                            'whip': whip,
                            'war': war
                        })
                if skip_salary:
                    print(f"선수 {player_name} 연봉 처리 스킵")
                    continue
            
                # 연봉 정보 로드 대기
                salary_link = driver.find_element(By.CSS_SELECTOR, '.p_won')
                salary_link.click()
                WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.table_type03')))
                salary_rows = driver.find_elements(By.CSS_SELECTOR, '.table_type03 tr')
                print(f"연봉 데이터 로드 완료")
                # 연봉을 연도별 통계에 맞춰서 추가
                for salary_row in salary_rows:
                    salary_columns = salary_row.find_elements(By.TAG_NAME, 'td')
                    if len(salary_columns) > 1:
                        salary_year = salary_columns[0].text.strip()  # 연도 가져오기 (불필요한 공백 제거)
                        salary_value = salary_columns[1].text.strip()  # 연봉 값 가져오기 (불필요한 공백 제거)
                        if not year_salary_data:
                               print("year_salary_data가 비어 있습니다.")  # 데이터가 없을 경우
                         # 연도별 통계와 연봉을 매칭하여 저장
                        for data in year_salary_data:
                            if data['year'] == salary_year:
                                data['salary'] = salary_value  # 해당 연도의 연봉을 추가
                                
                # 최종적으로 데이터를 CSV로 저장
                for data in year_salary_data:
                    writer.writerow([
                        team_name, player_name, data['year'], data['team_c'], data['age'], data['games'], data['gs'], data['gr'], 
                        data['gf'], data['cg'], data['sho'], data['w'], data['l'], data['s'], data['hd'], data['ip'], 
                        data['er'], data['r'], data['rra'], data['tbf'], data['h'], data['_2b'], data['_3b'], data['hr'], 
                        data['bb'], data['hp'], data['ib'], data['so'], data['roe'], data['bk'], data['wp'], data['era'], 
                        data['ra9'], data['rra9'], data['fip'], data['whip'], data['war'], data.get('salary', '')
                    ])
                    print(f"선수 {player_name} 데이터 저장 완료")
            except Exception as e:
                print(f"선수 {player_name} 처리 중 오류 발생")
# 드라이버 종료
driver.quit()

선수 문경찬 데이터 컬럼 불일치: 35개
선수 문경찬 연봉 처리 스킵
연봉 데이터 로드 완료
선수 임기준 데이터 저장 완료
선수 임기준 데이터 저장 완료
선수 임기준 데이터 저장 완료
선수 임기준 데이터 저장 완료
선수 임기준 데이터 저장 완료
선수 임기준 데이터 저장 완료
연봉 데이터 로드 완료
선수 고영창 데이터 저장 완료
선수 고영창 데이터 저장 완료
선수 고영창 데이터 저장 완료
선수 고영창 데이터 저장 완료
선수 고영창 데이터 저장 완료
연봉 데이터 로드 완료
선수 박진태 데이터 저장 완료
선수 박진태 데이터 저장 완료
선수 박진태 데이터 저장 완료
선수 박진태 데이터 저장 완료
연봉 데이터 로드 완료
선수 차명진 데이터 저장 완료
선수 차명진 데이터 저장 완료
선수 차명진 데이터 저장 완료
연봉 데이터 로드 완료
선수 윌랜드 데이터 저장 완료
연봉 데이터 로드 완료
선수 하준영 데이터 저장 완료
선수 하준영 데이터 저장 완료
선수 하준영 데이터 저장 완료
선수 하준영 데이터 저장 완료
선수 김세현 데이터 컬럼 불일치: 35개
선수 김세현 연봉 처리 스킵
연봉 데이터 로드 완료
선수 황인준 데이터 저장 완료
선수 황인준 데이터 저장 완료
선수 황인준 데이터 저장 완료
연봉 데이터 로드 완료
선수 김윤동 데이터 저장 완료
선수 김윤동 데이터 저장 완료
선수 김윤동 데이터 저장 완료
선수 김윤동 데이터 저장 완료
선수 김윤동 데이터 저장 완료
선수 양승철 중복 검색 완료
연봉 데이터 로드 완료
선수 양승철 데이터 저장 완료
선수 양승철 데이터 저장 완료
연봉 데이터 로드 완료
선수 김승범 데이터 저장 완료
선수 김승범 데이터 저장 완료
선수 황윤호 데이터 컬럼 불일치: 33개
선수 황윤호 연봉 처리 스킵
연봉 데이터 로드 완료
선수 변시원 데이터 저장 완료
선수 변시원 데이터 저장 완료
선수 변시원 데이터 저장 완료
선수 변시원 데이터 저장 완료
선수 변시원 데이터 저장 완료
선수 변시원 데이터 저장 완료
선수 변시원 데이터 저장 완료
선수 변시원 데이터